<a href="https://colab.research.google.com/github/NVREND/Coursera/blob/main/Capstone_NusaGo_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q tensorflow-recommenders

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
import tensorflow_recommenders as tfrs
from typing import Dict, Text

In [ ]:
# Import PyDrive and associated libraries.
# This only needs to be done once per notebook.
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import drive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Download a file based on its file ID.
#
# A file ID looks like: laggVyWshwcyP6kEI-y_W3P8D26sz
file_id_1 = '1kFmj22am8p8_PkXpZjiZeLrIc_2V4hmX'
download_1 = drive.CreateFile({'id': file_id_1})
download_1.GetContentFile('tourism_with_id.csv')

file_id_2 = '1hzr76LlqTeCziP34GAA57uQheva6Md7v'
download_2 = drive.CreateFile({'id': file_id_2})
download_2.GetContentFile('user.csv')

file_id_3 = '1kGo_JryF9qfcV_zeSmA6MNq5vQPDqtLC'
download_3 = drive.CreateFile({'id': file_id_3})
download_3.GetContentFile('tourism_rating.csv')

# Load data
rating = pd.read_csv("tourism_with_id.csv")
place = pd.read_csv("tourism_rating.csv")
user = pd.read_csv("user.csv")


In [ ]:
# Drop unnecessary columns
place = place.drop(["Unnamed: 11", "Unnamed: 12", "Time_Minutes"], axis=1)

In [ ]:
# Merge dataframes
merged_df = pd.merge(rating, place, how="outer", on="Place_Id")
merged_df = pd.merge(merged_df, user, how="outer", on="User_Id")

# Convert "User_Id" and "Place_Id" to strings
merged_df["User_Id"] = merged_df["User_Id"].astype(str)
merged_df["Place_Id"] = merged_df["Place_Id"].astype(str)

# Split the data
training_size = 0.8
rating_merge_place = merged_df.sample(frac=1).reset_index(drop=True)
train_df, test_df = train_test_split(rating_merge_place, test_size=1 - training_size)

In [ ]:
# Define the model
embedding_dimension = 32
user_model = tf.keras.Sequential(
    [
        tf.keras.layers.StringLookup(
            vocabulary=merged_df["User_Id"].unique(),
            mask_token=None,
            name="user_id_lookup",
        ),
        tf.keras.layers.Embedding(
            input_dim=len(merged_df["User_Id"].unique()) + 1,
            output_dim=embedding_dimension,
            name="user_embedding",
        ),
    ]
)

place_model = tf.keras.Sequential(
    [
        tf.keras.layers.StringLookup(
            vocabulary=merged_df["Place_Name"].unique(),
            mask_token=None,
            name="place_Name_lookup",
        ),
        tf.keras.layers.Embedding(
            input_dim=len(merged_df["Place_Name"].unique()) + 1,
            output_dim=embedding_dimension,
            name="place_embedding",
        ),
    ]
)

# Define the task
task = tfrs.tasks.Ranking(
    loss=tf.keras.losses.MeanSquaredError(),
    metrics=[tf.keras.metrics.RootMeanSquaredError()],
)


# Build the model
class RecommenderModel(tfrs.Model):
    def __init__(self, user_model, place_model, task):
        super().__init__()
        self.place_model: tf.keras.Model = place_model
        self.user_model: tf.keras.Model = user_model
        self.task: tf.keras.layers.Layer = task

    def compute_loss(
        self, features: Dict[Text, tf.Tensor], training=False
    ) -> tf.Tensor:
        user_embeddings = self.user_model(features["User_Id"])
        positive_place_embeddings = self.place_model(features["Place_Name"])
        return self.task(user_embeddings, positive_place_embeddings)


# Create the model instance
model = RecommenderModel(user_model, place_model, task)

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1))

# Create input pipelines
train_batch_size = 64
train_data = tf.data.Dataset.from_tensor_slices(dict(train_df))
train_data = train_data.batch(train_batch_size)

test_data = tf.data.Dataset.from_tensor_slices(dict(test_df))
test_data = test_data.batch(train_batch_size)

# Train the model
model.fit(train_data, epochs=5)

# Evaluate the model
model.evaluate(test_data)

Epoch 1/5
125/125 [==============================] - 1s 2ms/step - root_mean_squared_error: 0.0408 - loss: 0.0017 - regularization_loss: 0.0000e+00 - total_loss: 0.0017
Epoch 2/5
125/125 [==============================] - 0s 2ms/step - root_mean_squared_error: 0.0405 - loss: 0.0016 - regularization_loss: 0.0000e+00 - total_loss: 0.0016
Epoch 3/5
125/125 [==============================] - 0s 2ms/step - root_mean_squared_error: 0.0402 - loss: 0.0016 - regularization_loss: 0.0000e+00 - total_loss: 0.0016
Epoch 4/5
125/125 [==============================] - 0s 2ms/step - root_mean_squared_error: 0.0399 - loss: 0.0016 - regularization_loss: 0.0000e+00 - total_loss: 0.0016
Epoch 5/5
32/32 [==============================] - 0s 2ms/step - root_mean_squared_error: 0.0395 - loss: 0.0016 - regularization_loss: 0.0000e+00 - total_loss: 0.0016


[0.03951350599527359, 0.0016248244792222977, 0, 0.0016248244792222977]

In [ ]:
# Fungsi untuk mendapatkan rekomendasi tempat berdasarkan ID pengguna dan kategori
def get_place_recommendations(user_id, selected_category, k=5):
    # Ambil embedding dari model user untuk ID pengguna tertentu
    user_embedding = model.user_model(tf.constant([user_id]))

    # Ambil nama tempat unik dari dataframe tempat berdasarkan kategori yang dipilih
    places_in_category = merged_df[merged_df["Category"] == selected_category]
    unique_place_names = places_in_category["Place_Name"].unique()

    # Konversi nama tempat menjadi indeks menggunakan layer StringLookup
    place_indices = model.place_model.layers[0](tf.constant(unique_place_names))

    # Ambil embedding dari model tempat untuk semua tempat
    all_place_embeddings = model.place_model.layers[1](place_indices)

    # Hitung kesamaan (similarity) antara embedding pengguna dan embedding tempat
    similarity_scores = tf.matmul(
        user_embedding, all_place_embeddings, transpose_b=True
    )

    # Dapatkan indeks tempat dengan nilai similarity tertinggi
    top_indices = tf.argsort(tf.squeeze(similarity_scores), direction="DESCENDING")[:k]

    # Ambil nama tempat berdasarkan indeks
    top_place_names = unique_place_names[top_indices].tolist()

    return top_place_names


# Contoh penggunaan: Mendapatkan rekomendasi tempat untuk pengguna dengan ID tertentu dan kategori yang dipilih
user_id_to_recommend = "74"  # Ganti dengan ID pengguna yang ingin Anda rekomendasikan
selected_category = "Tempat Ibadah"  # Ganti dengan kategori yang dipilih oleh pengguna
recommendations = get_place_recommendations(user_id_to_recommend, selected_category)

print(
    f"Rekomendasi Tempat untuk Pengguna dengan ID {user_id_to_recommend} dan Kategori {selected_category}:"
)
for i, place_name in enumerate(recommendations, start=1):
    print(f"{i}. {place_name}")

Rekomendasi Tempat untuk Pengguna dengan ID 74 dan Kategori Tempat Ibadah:
1. Masjid Raya Bandung
2. Masjid Al-Imtizaj
3. Masjid Muhammad Cheng Hoo
4. Masjid Pusdai
5. Masjid Istiqlal


In [ ]:
# model.save('model.h5')